### 4. Реализация модели GPT 

В этой главе рассматриваются реализация большоай языковой модели (ЛЛМ):
- нормализация и активация
- короткие связи
- реализация преобразователя
- параметры модели 

![](images/llm4.1.png)

![](images/llm4.4.png)

### 4.1 Архитектура модели:
 
- токенизатор исходного текста ->  
- векторное представление токенов (параметр) ->  
- блок преобразователя (с механизмом внимания) ->  
- вектор вероятностей следующего токена (прогноз модели)

![](images/llm4.2.png)

![](images/llm4.3.png)

In [49]:
GPT_CONFIG_124M = {       # 124 млн параметров(?)
    "vocab_size": 50257,  # словарь БПЕ
    "context_length": 1024,      # число токенов на входе
    "emb_dim": 768,       # размерность векторного представления токенов
    "n_heads": 12,        # мультипликатор распараллеливания
    "n_layers": 12,       # число последовательных блоков преобразователя
    "drop_rate": 0.1,     # коэффициент прореживания
    "qkv_bias": False     # 
}

### 4.2 Нормализация, активация и скалярное произведение

Вспомним, что скалярное произведение векторных представлений токенов соответствует контекстной близости. Причем бОльшее значение означает бОльшую близость.  
Тогда в полученных значениях следует учитывать бОльшие значения и не учитывать мЕньшие. Реализующий этот принцип механизм заключается в использовании слоев нормализации и активации:
- нормализация приводит все значения к одному диапазону (-1, 1), явно определяя - что такое бОльшее и мЕньшее значение
- активация отбрасывает все (или почти все) значения меньше 0

Активация выбрасывает часть значений и сохраняет другие. Т.к. машинное обучение состоит из циклов, активацию можно представить как эволюционное сохранение сильных контекстных связей и отбрасывание слабых.  

![](images/llm4.5.png)

![](images/llm4.8.png)

In [50]:
# Пример нормализации для случайного тензора

import torch
from torch import nn

torch.manual_seed(123)  
batch_example = torch.randn(2, 5) 
layer = nn.Sequential(nn.Linear(5, 6), nn.ReLU())
out = layer(batch_example)
mean = out.mean(dim=-1, keepdim=True)
var = out.var(dim=-1, keepdim=True)
print('Выходные значения после активации: \n', out)
print("Медиана:\n", mean)
print("Дисперсия:\n", var)

out_norm = (out - mean) / torch.sqrt(var)
mean = out_norm.mean(dim=-1, keepdim=True)
var = out_norm.var(dim=-1, keepdim=True)
print("Нормализованные выходные значения:\n", out_norm)
print("Медиана:\n", mean)
print("Дисперсия:\n", var)

Выходные значения после активации: 
 tensor([[0.2260, 0.3470, 0.0000, 0.2216, 0.0000, 0.0000],
        [0.2133, 0.2394, 0.0000, 0.5198, 0.3297, 0.0000]],
       grad_fn=<ReluBackward0>)
Медиана:
 tensor([[0.1324],
        [0.2170]], grad_fn=<MeanBackward1>)
Дисперсия:
 tensor([[0.0231],
        [0.0398]], grad_fn=<VarBackward0>)
Нормализованные выходные значения:
 tensor([[ 0.6159,  1.4126, -0.8719,  0.5872, -0.8719, -0.8719],
        [-0.0189,  0.1121, -1.0876,  1.5173,  0.5647, -1.0876]],
       grad_fn=<DivBackward0>)
Медиана:
 tensor([[    0.0000],
        [    0.0000]], grad_fn=<MeanBackward1>)
Дисперсия:
 tensor([[1.0000],
        [1.0000]], grad_fn=<VarBackward0>)


In [51]:
# Нормализация в одном классе

class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))
        # смысл обучаемых параметров в нормализации?
        # self.scale = torch.ones(emb_dim)
        # self.shift = torch.zeros(emb_dim)
 
    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift

### 4.3 Поиск неявных связей (exploration of a richer representation space)

На рисунке в гл. 2, слова имели векторное представление размерности 2, т.е. матрица, строящая векторное представление токенов (обучаемый параметр!) суть классификатор, распределяющий все входящие слова по вероятностям принадлежности к одной или другой. Т.е. когда размерность векторного представления увеличивается - увеличивается сложность классификатора.  

Тогда последовательность операций:
1. увеличение размерности 
2. активация
3. уменьшение размерности до исходного

Можно представить как поиск неявных связей

![](images/llm4.9.png)

![](images/llm4.10.png)

In [52]:
# Вычислительно эффективное приближение GELU - 
# более гладкой и сложной, чем ReLU функции активации:

class GELU(nn.Module):
    def __init__(self):
        super().__init__()
 
    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) * 
            (x + 0.044715 * torch.pow(x, 3))
        ))

In [53]:
# Класс, выполняющий поиск неявных связей (с коэффициентом 4)

class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
            GELU(),
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
        )
 
    def forward(self, x):
        return self.layers(x)

### 4.4 Проблема исчезающих градиентов (vanishing gradients) и остаточные связи (residual / skip / short connections)

Алгоритм обратного распространения ошибки - способ обновления весов в обучаемых параметрах сети. Чем больше в сети слоев, т.е. чем она глубже - тем сложнее сохранить стабильность обучения, т.к. вычисляемые значения градиента после каждого слоя уменьшаются, по мере обратного распространения ошибки - от выхода сети к её входу (рисунок). Т.е. функция ошибки сильнее всего влияет на ближний к ней (близкий к выходу) слой, чем длиннее путь распространения ошибки - тем меньшее значение влияет на веса параметров.

Для минимизации этого эффекта используются остаточные связи - соединения, создающие дополнительный (более короткий) путь обработки данных. Метафорически это соответствует случаю, когда основной смысл прочитанного понят частично и после прочтения следующего предложения, необходимо вернуться к предыдущему (к его остаточному смыслу). 

![](images/llm4.12.png)


In [54]:
# Модель с пятью линейно-активационными блоками и параметром использовать / не использовать остаточные связи

class ExampleDeepNeuralNetwork(nn.Module):
    def __init__(self, layer_sizes, use_shortcut):
        super().__init__()
        self.use_shortcut = use_shortcut
        self.layers = nn.ModuleList([
            nn.Sequential(nn.Linear(layer_sizes[0], layer_sizes[1]), GELU()),
            nn.Sequential(nn.Linear(layer_sizes[1], layer_sizes[2]), GELU()),
            nn.Sequential(nn.Linear(layer_sizes[2], layer_sizes[3]), GELU()),
            nn.Sequential(nn.Linear(layer_sizes[3], layer_sizes[4]), GELU()),
            nn.Sequential(nn.Linear(layer_sizes[4], layer_sizes[5]), GELU())
        ])
 
    def forward(self, x):
        for layer in self.layers:
            layer_output = layer(x)
            # Реализация связи: к выходу слоя добавляется его вход
            if self.use_shortcut and x.shape == layer_output.shape:
                x = x + layer_output
            else:
                x = layer_output
        return x

In [55]:
# Функция печати значений градиентов слоев модели

def print_gradients(model, x):
    # Вычисляем выход модели
    output = model(x)
    # Берем какое-то целевое значение для примера
    target = torch.tensor([[0.]]) 
    # Вычисляем какую-то ошибку на основании целевого значения
    loss = nn.MSELoss()
    loss = loss(output, target)    
    # Применяем алгоритм обратного распространения ошибки
    loss.backward() 
    for name, param in model.named_parameters():
        if 'weight' in name:
            print('Слой ' f"{name} получил градиент с медианным значением: {param.grad.abs().mean().item()}")

In [56]:
# Задаем параметры слоев и входящий вектор

layer_sizes = [3, 3, 3, 3, 3, 1]  
sample_input = torch.tensor([[1., 0., -1.]])

# Пример вычислений градиентов модели, использующей остаточные связи

model_with_shortcut = ExampleDeepNeuralNetwork(
    layer_sizes, use_shortcut=True)
print_gradients(model_with_shortcut, sample_input)

# Пример вычислений градиентов модели, не использующей остаточные связи
print('----')
model_without_shortcut = ExampleDeepNeuralNetwork(
    layer_sizes, use_shortcut=False)
print_gradients(model_with_shortcut, sample_input)

Слой layers.0.0.weight получил градиент с медианным значением: 0.0056779212318360806
Слой layers.1.0.weight получил градиент с медианным значением: 0.015727611258625984
Слой layers.2.0.weight получил градиент с медианным значением: 0.010563054122030735
Слой layers.3.0.weight получил градиент с медианным значением: 0.010379952378571033
Слой layers.4.0.weight получил градиент с медианным значением: 0.19919389486312866
----
Слой layers.0.0.weight получил градиент с медианным значением: 0.011355842463672161
Слой layers.1.0.weight получил градиент с медианным значением: 0.03145522251725197
Слой layers.2.0.weight получил градиент с медианным значением: 0.02112610824406147
Слой layers.3.0.weight получил градиент с медианным значением: 0.020759904757142067
Слой layers.4.0.weight получил градиент с медианным значением: 0.3983877897262573


### 4.5 Реализация блока преобразователя (transformer block)

Идея преобразователя состоит в объединении механизма внимания (поиск явных связей во входящей последовательности) и механизма поиска неявных связей.  

Итоговая архитектура является результатом практического опыта и на разных этапах могла отличаться. Так, ранее нормализация использовалась после указанных механизмов (Post-LayerNorm), что оказалось менее эффективно чем настоящая версия (Pre-LayerNorm).

![](images/llm4.13.png)


In [57]:
# Копия из предыдущей главы для корректного запуска кода

class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, 
                 context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert d_out % num_heads == 0, "d_out must be divisible by num_heads"
        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            'mask',
             torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )
 
    def forward(self, x):
        b, num_tokens, d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
 
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)
 
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)
 
        attn_scores = queries @ keys.transpose(2, 3)
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
  
        attn_scores.masked_fill_(mask_bool, -torch.inf) 
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)
 
        context_vec = (attn_weights @ values).transpose(1, 2)
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec)
        return context_vec

In [58]:
# Реализация блока преобразователя
 
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"], 
            dropout=cfg["drop_rate"],
            qkv_bias=cfg["qkv_bias"])
        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_resid = nn.Dropout(cfg["drop_rate"])
 
    def forward(self, x):
        
        # вход сохранен для передачи остатка
        shortcut = x
        # вход нормализован
        x = self.norm1(x)
        # механизм внимания
        x = self.att(x)
        # прореживание
        x = self.drop_resid(x)
        # передача остатка к выходу прореживания
        x = x + shortcut  # Add the original input back
 
        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_resid(x)
        x = x + shortcut
        return x

### 4.6 Реализация модели ГПТ 

В этой архитектуре, изменив конфигурационный блок (cfg), можно масштабировать модель. Изучались следующие соотношения размерности пространства токенов / числа блоков преобразователя / коэффициента параллелизма (число "голов" в механизме внимания):
- малый:            768 / 12 / 12 (рисунок)
- средний:          1024 / 24 / 16
- большой:          1280 / 36 / 20
- очень большой:    1600 / 48 / 25

![](images/llm4.15.png)

In [59]:
# Реализация модели ГПТ-2

class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])
        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])
        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )
 
    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

In [60]:
# Подсчитаем число параметров модели:

model = GPTModel(GPT_CONFIG_124M)
total_params = sum(p.numel() for p in model.parameters())
print(f"Общее число параметров: {total_params:,}")

Total number of parameters: 163,009,536


In [62]:
# Это число отличается от заявленных 124 млн, т.к. в модели ГПТ-2 для сокращения вычислений (weight tying)
# для построения векторного представления токенов  и для построения выходного слоя вероятностей
# использовался один и тот же слой, размером 39 млн параметров

print("Размерность слоя векторного представления:", model.tok_emb.weight.shape)
print("Размерность выходного слоя:", model.out_head.weight.shape)

total_params_gpt2 =  total_params - sum(p.numel() for p in model.out_head.parameters())
print(f"Число параметров модели с учетом сокращения вычислений: {total_params_gpt2:,}")

Размерность слоя векторного представления: torch.Size([50257, 768])
Размерность выходного слоя: torch.Size([50257, 768])
Число параметров модели с учетом сокращения вычислений: 124,412,160


### 4.7 Генерация текста без обучения

![](images/llm4.17.png)

In [63]:
# Функция генерации текста на основе модели
# параметром idx задается входная строка
# параметром max_new_tokens ограничиывается длина выходной строки

def generate_text_simple(model, idx, max_new_tokens, context_size):
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -context_size:]
        with torch.no_grad():
            logits = model(idx_cond)
       
        logits = logits[:, -1, :]
        probas = torch.softmax(logits, dim=-1)
        idx_next = torch.argmax(probas, dim=-1, keepdim=True)
        idx = torch.cat((idx, idx_next), dim=1)
 
    return idx

In [65]:
# Токенизация 
import tiktoken
 
tokenizer = tiktoken.get_encoding("gpt2")
start_context = "Hello, I am"
encoded = tokenizer.encode(start_context)
print("encoded:", encoded)
encoded_tensor = torch.tensor(encoded).unsqueeze(0)

encoded: [15496, 11, 314, 716]


In [66]:
# Выход модели в токенах
model.eval()
out = generate_text_simple(
    model=model,
    idx=encoded_tensor, 
    max_new_tokens=6, 
    context_size=GPT_CONFIG_124M["context_length"]
)
print("Output:", out)

Output: tensor([[15496,    11,   314,   716, 24922, 20900, 45222, 37903, 18909, 17586]])
Output length: 10


In [68]:
# Детокенизация (без обучения выдан мусор)

decoded_text = tokenizer.decode(out.squeeze(0).tolist())
print(decoded_text)

Hello, I am Opportuninator classmate Sioux Cele overturn


### 4.8 Резюме

- Нормализация слоев стабилизирует обучения, гарантируя, что выходные данные каждого слоя имеют одинаковое среднее значение и дисперсию (измеряются в одних цифрах)
- Короткие связи - это соединения, которые пропускают один или несколько слоев, передавая выходные данные одного слоя непосредственно на более глубокий уровень (решение проблемы исчезающих градиентов)
- Блок преобразователя - основной элемент ЛЛМ, ищущий связи между токенами
- Модели ГПТ архитектурно одинаковы, отличаются числом блоков преобразователя, размерностью пространства токенов и коэффициентом параллелизма
- Без обучения ЛЛМ генерирует бессвязный текст (текст со случайными связями)